# Investigation of single token prediction in Pythia 160m

Investigating how the Pythia 160m model represents knowledge from pre-training, using a prompt intended to generate a token for Dublin as the capital of Ireland. This is the smallest model which behaves like this.

In [1]:
# Install dependencies and set up the package
# %pip install transformers torch matplotlib

# Install the local package in development mode
import sys
sys.path.insert(0, '../src')

In [2]:
from transformer_algebra import PromptedTransformer, load_pythia_model, logits

# Load Pythia-160m-deduped from HuggingFace
model, tokenizer = load_pythia_model("EleutherAI/pythia-160m-deduped")
print(f"Model: {model.config.name_or_path}")
print(f"Layers: {model.config.num_hidden_layers}")
print(f"Heads: {model.config.num_attention_heads}")
print(f"d_model: {model.config.hidden_size}")

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model: EleutherAI/pythia-160m-deduped
Layers: 12
Heads: 12
d_model: 768


In [3]:
T = PromptedTransformer(model, tokenizer, "The capital of Ireland")
T  # Displays as T(<4 tokens>)

T(<4 tokens>)

In [4]:
x = T(" is")
x  # Displays as x^12[' is'] after T('The capital of...')

x^12[' is'] after T('The  capital  of...')

In [5]:
type(x)

transformer_algebra.core.ResidualVector

In [9]:
#predict(x)
# Should return an object representing a mapping from tokens to probabilities
#logits(x) 
# Should return an object representing a mapping from tokens to logits
#summarise(logits(x)) - outputs as below
L = logits(x)
L

logits(x^12[' is'] after T('The  capital  of...'))

In [10]:
L.summary()

"Top 5 predictions:\n  1. ' the' (logit: 790.24)\n  2. ' a' (logit: 789.05)\n  3. ' in' (logit: 789.05)\n  4. ' now' (logit: 788.15)\n  5. ' to' (logit: 787.84)"

In [11]:
L[" Dublin"]

<' Dublin', x^12[' is'] after T('The  capital  of...')> = 786.74

Expected result: Top 5 predictions after final layer:
  1. ' the' (logit: 786.61)
  2. ' a' (logit: 784.93)
  3. ' Belfast' (logit: 784.57)
  4. ' Dublin' (logit: 784.47)
  5. ' now' (logit: 784.44)

In [ ]:
#expand(x)
# Should return an object representing the decomposition of the residual stream at x into contributions from each layer
#Something like:
# LN( \underline{ is} + \T_0 \underline{ is} +  \T_1  \underline{ is} ... )

The next step depends on a relation:
$$
<x, LN(a + b)> = \frac{<x,a>+<x,b>}{\left \| a+b \right \|}
$$

In [26]:
#logits(x)["Dublin"]
# Should return something like
# < \overline{Dublin}, T(\underline{ is}) > = 783.45


In [27]:
#expand(logits(x)["Dublin"])
# Should return something like
# scale * ( < \overline{Dublin}, T_0(\underline{ is} > + < \overline{Dublin}, T_1(\underline{ is} > + ... ) = 783.45
